In [ ]:
import pandas as pd
import numpy as np
import random
import glob
import tempfile
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn, optim, Tensor
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchinfo import summary
import timm
import torchvision
from torchvision import transforms, datasets, models
import torch.utils.data as Data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv(r"/kaggle/input/UBC-OCEAN/train.csv")
test = pd.read_csv(r"/kaggle/input/UBC-OCEAN/test.csv")
train_tma = train[train["is_tma"] == True]
train_no_tma = train[train["is_tma"] == False]

In [ ]:
train_tma['img_id_ext'] = [str(i) + ".png" for i in train_tma['image_id']]
train_no_tma['img_id_ext'] = [str(i) + "_thumbnail.png" for i in train_no_tma['image_id']]
test['img_id_ext'] = [str(i) + "_thumbnail.png" for i in test['image_id']]

In [ ]:
train_df = pd.concat([train_tma, train_no_tma])
train_df.sort_index(ascending = True, inplace = True)
train_df.head()

In [ ]:
le = LabelEncoder()
image_label = train_df['label']
le.fit(image_label)
train_df['label'] = le.transform(image_label)
train_df.head()

In [ ]:
# import os

# class TrainDataset(Dataset):
#     def __init__(self, df, root_path, train_thumbnails, train_images, transform = None):
#         self.df = df
#         self.root_path = root_path
#         self.train_thumbnails = train_thumbnails
#         self.train_images = train_images
#         self.transform = transform
        
#     def __len__(self):
#         return len(self.df)
    
#     def __getitem__(self, idx):
#         img_name = self.df.iloc[idx, -1]
        
#         if "thumbnail" in img_name:
#             img_path = os.path.join(root_path + train_thumbnails, img_name)
#         else:
#             img_path = os.path.join(root_path + train_images, img_name)
        
#         img = Image.open(img_path).convert('RGB')
        
#         if self.transform:
#             img = self.transform(img)
        
#         label = self.df.iloc[idx, 1]
#         label = torch.tensor(label)
#         return img, label

# norm_mean = [0.485, 0.456, 0.406]
# norm_std = [0.229, 0.224, 0.225]

# train_transform = transforms.Compose([
#     transforms.Resize((224, 224)), 
#     transforms.CenterCrop(224), 
#     transforms.RandomHorizontalFlip(p = 0.5), 
#     transforms.RandomVerticalFlip(p = 0.5),
#     transforms.ToTensor(),
#     transforms.Normalize(norm_mean, norm_std),
# ])

# valid_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(norm_mean, norm_std),
# ])

# root_path = "/kaggle/input/UBC-OCEAN/"
# train_thumbnails = "train_thumbnails/"
# train_images = "train_images/"
# test_thumbnails = "test_thumbnails/"
# test_images = "test_images/"

# dataset = TrainDataset(df = train_df, root_path = root_path, train_thumbnails = train_thumbnails, train_images = train_images, transform = train_transform)

# train_ratio = 0.8
# valid_ratio = 0.2

# train_size = int(train_ratio * len(train_df))
# valid_size = len(train_df) - train_size

# dataset_train, dataset_valid = random_split(dataset, [train_size, valid_size])

# train_dataloader = DataLoader(dataset_train, batch_size = 8, shuffle = False, num_workers = 5)
# valid_dataloader = DataLoader(dataset_valid, batch_size = 8, shuffle = False, num_workers = 5)

# dataiter = iter(train_dataloader)
# images = next(dataiter)
# images[0].shape

In [ ]:
import os

class TrainDataset(Dataset):
    def __init__(self, df, root_path, train_thumbnails, train_images, transform=None):
        self.df = df
        self.root_path = root_path
        self.train_thumbnails = train_thumbnails
        self.train_images = train_images
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, -1]
        
        if "thumbnail" in img_name:
            img_path = os.path.join(self.root_path + self.train_thumbnails, img_name)
        else:
            img_path = os.path.join(self.root_path + self.train_images, img_name)
        
        img = Image.open(img_path).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
        
        label = self.df.iloc[idx, 1]
        label = torch.tensor(label)
        
        return img, label, img_name  # Return img_name along with img and label


norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.CenterCrop(224), 
    transforms.RandomHorizontalFlip(p = 0.5), 
    transforms.RandomVerticalFlip(p = 0.5),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

root_path = "/kaggle/input/UBC-OCEAN/"
train_thumbnails = "train_thumbnails/"
train_images = "train_images/"
test_thumbnails = "test_thumbnails/"
test_images = "test_images/"

dataset = TrainDataset(df=train_df, root_path=root_path, train_thumbnails=train_thumbnails, train_images=train_images, transform=train_transform)

train_ratio = 0.8
valid_ratio = 0.2

train_size = int(train_ratio * len(train_df))
valid_size = len(train_df) - train_size

dataset_train, dataset_valid = random_split(dataset, [train_size, valid_size])

train_dataloader = DataLoader(dataset_train, batch_size = 8, shuffle = False, num_workers = 5)
valid_dataloader = DataLoader(dataset_valid, batch_size = 8, shuffle = False, num_workers = 5)

dataiter = iter(train_dataloader)
images = next(dataiter)
images[0].shape

dataiter = iter(valid_dataloader)
images, labels, image_names = next(dataiter)
# Now, image_names contains the file names of the images in the validation set
image_names

In [ ]:
def show_pic(dataloader):
    examples = enumerate(dataloader)
    batch_idx, (example_data, example_targets) = next(examples)
    classes = ('HGSC', 'LGSC', 'EC', 'CC', 'MC')
    fig = plt.figure()
    for i in range(5):
        plt.subplot(2, 3, i + 1)
        img = example_data[i]
        print('pic shape:', img.shape)
        img = img.swapaxes(0, 1)
        img = img.swapaxes(1, 2)
        plt.imshow(img, interpolation = 'none')
        plt.title(classes[example_targets[i].item()])
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [ ]:
def build_model(num_classes, model_type = "resnet18", freeze_convnets = False, single_layer = True, hidden_units = 32):
    # Load pre_trained model
    if model_type == "resnet18":
        model = models.resnet18(pretrained = True)
    elif model_type == "resnet50":
        model = models.resnet50(pretrained = True)
    elif model_type == "resnet101":
        model = models.resnet101(pretrained = True)
    elif model_type == "resnet152":
        model = models.resnet152(pretrained = True)
    else:
        model = models.resnet18(pretrained = True)
        
    # Freeze the convnets
    if freeze_convnets:
        for param in model.parameters():
            param.requires_grad = false
    
    # Get the output dimension from the Conv block
    num_ftrs = model.fc.in_features
    
    # Parameters of newly constructed modules have requires_grad + True by default
    # Here the size of each output sample is set to num_classes
    if single_layer:
        model.fc = nn.Linear(num_ftrs, num_classes)
    else:
        model.fc = nn.Sequential(
                    nn.Linear(num_ftrs, hidden_units),
                    nn.ReLU(),
                    nn.Linear(hidden_units, num_classes)
        )
    return model

In [ ]:
def train(net, loss, train_dataloader, valid_dataloader, device, batch_size, num_epoch, lr, lr_min, optim = 'sgd', init = True, scheduler_type = 'Cosine'):
    def init_xavier(m):
        if type(m) == nn.Linear:
            nn.init.xavier_normal_(m.weight)
    
    if init:
        net.apply(init_xavier)
    
    print('training on:', device)
    net.to(device)
    
    if optim == 'sgd':
        optimizer = torch.optim.SGD((param for param in net.parameters() if param.requires_grad), lr = lr, weight_decay = 0)
    elif optim == 'adam':
        optimizer = torch.optim.Adam((param for param in net.parameters() if param.requires_grad), lr = lr, weight_decay = 1e-3)
    elif optim == 'adamW':
        optimizer = torch.optim.AdamW((param for param in net.parameters() if param.requires_grad), lr = lr, weight_decay = 0)
    elif optim == 'ranger':
        optimizer = torch.optim.Ranger((param for param in net.parameters() if param.requires_grad), lr = lr, weight_decay = 0)
    
    if scheduler_type == 'Cosine':
        scheduler = CosineAnnealingLR(optimizer, T_max = num_epoch, eta_min = lr_min)
    
    train_losses = []
    train_access = []
    eval_access = []
    best_acc = 0.0
    
    # Train
    for epoch in range(num_epoch):
        print("------Start of training round {}------".format(epoch + 1))
        
        net.train()
        train_acc = 0
        
        for batch in tqdm(train_dataloader, desc = 'Train'):
            imgs, targets = batch
            imgs = imgs.to(device)
            targets = targets.to(device)
            output = net(imgs)
            
            Loss = loss(output, targets)
            
            optimizer.zero_grad()
            Loss.backward()
            optimizer.step()
            
            _, pred = output.max(1)
            num_correct = (pred == targets).sum().item()
            acc = num_correct / (batch_size)
            train_acc += acc
        
        scheduler.step()
        print("epoch: {}, Loss: {}, Acc: {}".format(epoch, Loss.item(), train_acc / len(train_dataloader)))
        train_access.append(train_acc / len(train_dataloader))
        train_losses.append(Loss.item())
        
        net.eval()
        eval_loss = 0
        eval_acc = 0
        with torch.no_grad():
            for imgs, targets in valid_dataloader:
                imgs = imgs.to(device)
                targets = targets.to(device)
                output = net(imgs)
                Loss = loss(output, targets)
                _, pred = output.max(1)
                num_correct = (pred == targets).sum().item()
                eval_loss += Loss
                acc = num_correct / imgs.shape[0]
                eval_acc += acc
                
            eval_losses = eval_loss / (len(valid_dataloader))
            eval_acc = eval_acc / (len(valid_dataloader))
            if eval_acc > best_acc:
                best_acc = eval_acc
                torch.save(net.state_dict(), 'best_acc.pth')
            eval_access.append(eval_acc)
            print("Loss on the overall validation set: {}".format(eval_losses))
            print("Correctness on the overall validation set: {}".format(eval_acc))
        
    return train_losses, train_access, eval_access

In [ ]:
def show_access(train_losses, train_access, valid_access, num_epoch):
    plt.plot(1 + np.arange(len(train_losses)), train_losses, linewidth = 1.5, linestyle = 'dashed', label = 'train_losses')
    plt.plot(1 + np.arange(len(train_access)), train_access, linewidth = 1.5, linestyle = 'dashed', label = 'train_access')
    plt.plot(1 + np.arange(len(eval_access)), eval_access, linewidth = 1.5, linestyle = 'dashed', label = 'eval_access')
    plt.grid()
    plt.xlabel('epoch')
    plt.xticks(range(1, 1 + num_epoch, 1))
    plt.legend()
    plt.show()

In [ ]:
show_pic(train_dataloader)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
net = build_model(num_classes = 5, model_type = "resnet50", freeze_convnets = False, single_layer = True, hidden_units = 64)

In [ ]:
loss = nn.CrossEntropyLoss()

In [ ]:
train_losses, train_access, eval_access = train(net, loss, train_dataloader, valid_dataloader, device, batch_size = 8, num_epoch = 20, lr = 3e-4, lr_min = 1e-4, optim = 'adam', init = False)

In [ ]:
classes = ('HGSC', 'LGSC', 'EC', 'CC', 'MC')
image_id = []
label = []

with torch.no_grad():
    for imgs, targets, name in valid_dataloader:
        imgs = imgs.to(device)
        targets = targets.to(device)
        output = net(imgs)
        _, pred = output.max(1)
        pred = pred.tolist()
        targets = targets.tolist()        
        for idx, _ in enumerate(pred):
            score = pred[idx]
            target = targets[idx]
            print(name[idx], classes[score], classes[target])
        break
        
